In [69]:
!pip install pandas

In [70]:
import pandas as pd

# Load the TSV file into a DataFrame
file_path = './supplier_questions.tsv' # change to your questions file
df = pd.read_csv(file_path, sep='\t')

# Check the content of the DataFrame (just to ensure it's loaded correctly)
df.head()

# concat the questions in the DataFrame
concatenated_questions = "\n".join(str(question) for question in df['Question'])
print("concatenated_questions:\n" + concatenated_questions)


concatenated_questions:
Is the supplier ISO 27001 certified?
Is the supplier SOC2 certified?


In [71]:
# Load the TSV file into a DataFrame
file_path = './context.tsv' # change to your resource file
df = pd.read_csv(file_path, sep='\t')

# Check the content of the DataFrame (just to ensure it's loaded correctly)
df.head()

# concat the questions in the DataFrame
concatenated_resources = "\n".join(str(res) for res in df['Resources'])
print("concatenated_resources:\n" + concatenated_resources)

concatenated_resources:
https://sentry.io/legal/dpa/5.1.0/in-app/?userCurrentVersion=5.1.0
https://sentry.io/security/
https://sentry.io/legal/terms/3.0.0/in-app/?userCurrentVersion=3.0.1


In [72]:
base_prompt = """
Below is a list of questions and resources.
Please answer the questions based on the resources provided:
"""

In [73]:
prompt = base_prompt + "\nQuestions:\n" + concatenated_questions + "\n\nResources:\n" + concatenated_resources
print("prompt:\n" + prompt)

prompt:

Below is a list of questions and resources.
Please answer the questions based on the resources provided:

Questions:
Is the supplier ISO 27001 certified?
Is the supplier SOC2 certified?

Resources:
https://sentry.io/legal/dpa/5.1.0/in-app/?userCurrentVersion=5.1.0
https://sentry.io/security/
https://sentry.io/legal/terms/3.0.0/in-app/?userCurrentVersion=3.0.1


In [107]:
anthropic_system = """
You're an ISO leading a compliance team seeking information about a potential future supplier. 
Your team is asked to perform a compliance due diligence. 
Respond in short and clear explanations and including URL resources if available.
Don't be lazy!
"""

In [108]:
!pip install anthropic
!pip install python-dotenv

In [109]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('ANTHROPIC_API_KEY')
if api_key is None:
    raise ValueError("The 'ANTHROPIC_API_KEY' environment variable is not set.")

In [110]:
import anthropic
client = anthropic.Anthropic()

def ask(prompt):
    response = client.messages.create(
        model="claude-3-5-sonnet-latest",
        max_tokens=2000,
        temperature=0.0,
        system=anthropic_system,
        messages=[{"role": "user", "content": prompt}]
    )
    # Extract and return the text content
    if response.content:
        text_response = response.content[0].text  # Assuming `response.content` is a list with a `TextBlock`
        return text_response
    else:
        print("Empty response.content")
        return ""

In [111]:
import json

try:
    res = ask(prompt)
    if not res.strip():  # Handle empty responses
        print("Empty response, exiting.")
    else:
        print("result:\n" + str(res))

except Exception as e:
    print(f"Exception occurred: {e}")
    print(f"Exception type: {type(e)}")


result:
Based on the provided resources, specifically reviewing Sentry's security page (https://sentry.io/security/):

ISO 27001:
Yes, Sentry is ISO 27001:2013 certified. This certification covers their information security management system (ISMS).

SOC 2:
Yes, Sentry maintains SOC 2 Type II compliance. They undergo regular audits to maintain this certification.

Additional note: You can request access to their compliance reports (ISO 27001 certificate and SOC 2 report) by contacting their security team at security@sentry.io or through their security portal at https://sentry.io/security/

These certifications indicate that Sentry has implemented robust security controls and undergoes regular third-party audits to verify their security practices.
